# Heiken Ashi Trading Strategy
This notebook implements and backtests a trading strategy based on **Heiken Ashi Indicators**.

### Strategy Logic:
- **Buy (Long Entry)**: On the first green Heiken Ashi candle (HA Close > HA Open).
- **Sell (Exit)**: On the first red Heiken Ashi candle (HA Close < HA Open) following a buy.

We use the project's built-in modular backtester with `strict_signals=True` to simulate discrete entry and exit events.


## 0) Setup
Imports from the local `src` directory.


In [1]:
import os, sys
import numpy as np
import pandas as pd
from bokeh.io import output_notebook, show

# --- Setup correct working directory (ROOT) ---
if os.getcwd().endswith('notebooks'):
    os.chdir('..')

ROOT = os.getcwd()
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

from src.backtester.data import load_cleaned_assets, align_close_prices
from src.backtester.engine import BacktestConfig, run_backtest
from src.backtester.metrics import compute_performance_stats
from src.backtester.models import HeikenAshiMicroModel, EqualWeightAllocator, MPTAllocator, combine_models_to_weights
from src.backtester.bokeh_plots import build_interactive_portfolio_layout
from src.backtester.report import compute_backtest_report

output_notebook()


Loading BokehJS ...

## 0.1) Backtest Configuration
We enable `strict_signals=True` for event-driven entry/exit simulations.


In [2]:
cfg = BacktestConfig(
    initial_equity=1_000_000,
    transaction_cost_bps=5,
    rebalance=None,  # Trade on signal changes
    mode='event_driven',
    strict_signals=True,
    stop_loss_pct=0.0
)
cfg


BacktestConfig(initial_equity=1000000, transaction_cost_bps=5, rebalance=None, allow_leverage=False, mode='event_driven', trade_buffer=0.0, no_sell=False, strict_signals=True, stop_loss_pct=0.0, stop_loss_type='trailing')

## 1) Data Loading


In [3]:
assets = load_cleaned_assets(symbols=None)
close = align_close_prices(assets)
open_prices = pd.concat([df['Open'].astype(float).rename(s) for s, df in assets.items()], axis=1).sort_index()
high_prices = pd.concat([df['High'].astype(float).rename(s) for s, df in assets.items()], axis=1).sort_index()
low_prices = pd.concat([df['Low'].astype(float).rename(s) for s, df in assets.items()], axis=1).sort_index()
close.tail()


,Asset_001,Asset_002,Asset_003,Asset_004,Asset_005,Asset_006,Asset_007,Asset_008,Asset_009,Asset_010,...,Asset_091,Asset_092,Asset_093,Asset_094,Asset_095,Asset_096,Asset_097,Asset_098,Asset_099,Asset_100
Date,,,,,,,,,,,,,,,,,,,,,
2026-01-12,331.752326,661.575709,342.250250,281.999689,242.195518,820.121795,227.300860,621.365964,933.522455,219.746103,...,41.365796,271.489926,198.902189,113.998969,295.600064,224.360012,337.968596,100.955128,604.306539,25.913585
2026-01-13,332.772106,652.550091,346.056097,277.571818,243.334854,806.222579,226.409812,617.091996,930.232191,209.936920,...,42.052935,269.856217,201.506879,116.810018,283.866464,226.316808,330.405426,102.224305,617.759396,26.215247
2026-01-14,331.382639,636.897307,345.932669,270.764086,239.838252,786.331770,222.359547,614.487763,926.864341,210.762894,...,43.033254,277.756922,206.596156,119.729182,287.383561,228.194265,325.734626,104.183527,615.993968,26.619917
2026-01-15,329.151836,633.126221,342.692546,272.514641,244.958753,793.076972,222.040587,613.827360,892.045017,209.853679,...,42.666778,279.002299,204.614479,116.788392,285.348139,230.838565,328.939693,103.924754,618.152582,26.708207
2026-01-16,325.735529,637.562765,339.791851,273.590149,243.884880,792.374358,221.498860,614.662228,896.734265,210.205829,...,42.813368,277.341803,209.801342,114.954705,278.852755,232.231218,354.462977,104.146552,618.121761,26.671420


## 2) Market Proxy for Comparison
We compute a market proxy OHLCV for the Bokeh visualization.


In [4]:
def build_market_proxy_ohlcv(assets, index):
    opens = pd.concat([df['Open'].astype(float).reindex(index) for df in assets.values()], axis=1).mean(axis=1)
    highs = pd.concat([df['High'].astype(float).reindex(index) for df in assets.values()], axis=1).mean(axis=1)
    lows = pd.concat([df['Low'].astype(float).reindex(index) for df in assets.values()], axis=1).mean(axis=1)
    closes = pd.concat([df['Close'].astype(float).reindex(index) for df in assets.values()], axis=1).mean(axis=1)
    return pd.DataFrame({'Open': opens, 'High': highs, 'Low': lows, 'Close': closes})

market_df = build_market_proxy_ohlcv(assets, close.index)
market_df.tail()


,Open,High,Low,Close
Date,,,,
2026-01-12,316.717404,319.981102,312.805504,317.286005
2026-01-13,317.039750,319.475733,311.476075,314.856757
2026-01-14,313.361236,316.951272,309.083688,313.861488
2026-01-15,314.961632,318.647916,311.134632,315.041594
2026-01-16,315.422198,318.202870,312.049382,314.392333


## 3) Heiken Ashi Strategy Implementation
We use the `HeikenAshiMicroModel` which calculates HA candles and discrete signals internally.


In [5]:
model = HeikenAshiMicroModel()
# We scale signals by 0.1 to allocate 10% of portfolio per asset if multiple assets trigger
raw_signals = model.compute_signals(assets)
signals = raw_signals * 0.16 # Example allocation
signals.tail()


,Asset_001,Asset_002,Asset_003,Asset_004,Asset_005,Asset_006,Asset_007,Asset_008,Asset_009,Asset_010,...,Asset_091,Asset_092,Asset_093,Asset_094,Asset_095,Asset_096,Asset_097,Asset_098,Asset_099,Asset_100
Date,,,,,,,,,,,,,,,,,,,,,
2026-01-12,-0.16,-0.16,0.16,0.16,-0.16,-0.16,0.16,-0.16,-0.16,-0.16,...,0.16,-0.16,-0.16,0.16,0.16,0.16,0.16,0.16,0.16,-0.16
2026-01-13,0.16,-0.16,0.16,-0.16,0.16,-0.16,0.16,-0.16,0.16,-0.16,...,0.16,-0.16,0.16,0.16,-0.16,0.16,0.16,0.16,0.16,0.16
2026-01-14,-0.16,-0.16,0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,...,0.16,0.16,0.16,0.16,-0.16,0.16,-0.16,0.16,0.16,0.16
2026-01-15,-0.16,-0.16,0.16,-0.16,0.16,-0.16,-0.16,-0.16,-0.16,-0.16,...,0.16,0.16,0.16,0.16,-0.16,0.16,0.16,0.16,0.16,0.16
2026-01-16,-0.16,-0.16,-0.16,-0.16,0.16,-0.16,-0.16,-0.16,-0.16,-0.16,...,0.16,0.16,0.16,-0.16,-0.16,0.16,0.16,0.16,0.16,0.16


## 4) Backtest Execution


In [6]:
print('Running Heiken Ashi Strategy...')
res = run_backtest(
    close_prices=close, 
    weights=signals, 
    config=cfg, 
    open_prices=open_prices,
    high_prices=high_prices,
    low_prices=low_prices
)
report = compute_backtest_report(result=res, close_prices=close, benchmark='equal_weight')
display(report)


Running Heiken Ashi Strategy...


Start                         2016-01-25 00:00:00
End                           2026-01-16 00:00:00
Duration                       3644 days 00:00:00
Initial Equity                          1000000.0
Final Equity                        3442629.38495
Equity Peak                        3558588.096368
Total Return [%]                       244.262938
CAGR [%]                                13.214728
Volatility (ann) [%]                    18.964477
Sharpe                                   0.749488
Sortino                                  1.178069
Max Drawdown [%]                       -37.680529
Calmar                                   0.350704
Best Day [%]                            10.973993
Worst Day [%]                          -12.142291
Avg Gross Exposure                       0.986287
Avg Net Exposure                         0.986287
Exposure Time [%]                       99.721227
Rebalance Days                               2123
Total Turnover                  2696623201.695777


## 5) Interactive Bokeh Visualization


In [7]:
layout = build_interactive_portfolio_layout(
    market_ohlcv=market_df,
    equity=res.equity,
    returns=res.returns,
    weights=res.weights,
    turnover=res.turnover,
    costs=res.costs,
    close_prices=close, 
    title='Heiken Ashi Strategy'
)
show(layout)
